# InfluxDB Simple Testing

Quick and easy InfluxDB testing for AWS Timestream for InfluxDB.

**What you need:**
- InfluxDB Token (see `GET_INFLUXDB_TOKEN.md` for instructions)
- Your InfluxDB endpoint URL
- Organization name and bucket name

**What this does:**
- ✅ Test connection to InfluxDB
- ✅ Write sample energy metrics
- ✅ Write sample sensor data
- ✅ Query and visualize data
- ✅ Simple and focused (no AWS Bedrock or other services)

## Step 1: Install Required Packages

In [ ]:
# Install packages (run this once)
!pip install -q influxdb-client pandas plotly

## Step 2: Import Libraries

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, HTML

print("✓ Libraries imported successfully")

## Step 3: Configure Your InfluxDB Connection

**⚠️ IMPORTANT: Enter your InfluxDB token below**

See `GET_INFLUXDB_TOKEN.md` for instructions on how to get your token.

In [ ]:
# ============================================
# CONFIGURATION - Edit these values
# ============================================

# Your InfluxDB token (paste it here)
INFLUXDB_TOKEN = ""  # ⬅️ PASTE YOUR TOKEN HERE

# Your InfluxDB endpoint
INFLUXDB_URL = "https://Lhk52q7uoe-lktzzbuyksah47.timestream-influxdb.us-east-1.on.aws"

# Organization and bucket
INFLUXDB_ORG = "superapp-org"
INFLUXDB_BUCKET = "test-bucket"  # Change this if using a different bucket

# Database name (for reference)
DATABASE_NAME = "superAppDB"

# ============================================

# Validation
if not INFLUXDB_TOKEN:
    print("❌ ERROR: Please set your INFLUXDB_TOKEN above!")
    print("   See GET_INFLUXDB_TOKEN.md for instructions")
else:
    print("✓ Configuration set")
    print(f"  URL: {INFLUXDB_URL}")
    print(f"  Org: {INFLUXDB_ORG}")
    print(f"  Bucket: {INFLUXDB_BUCKET}")
    print(f"  Database: {DATABASE_NAME}")
    print(f"  Token: {'*' * 20}... (hidden)")

## Step 4: Connect to InfluxDB

In [ ]:
# Create InfluxDB client
try:
    client = InfluxDBClient(
        url=INFLUXDB_URL,
        token=INFLUXDB_TOKEN,
        org=INFLUXDB_ORG,
        timeout=30_000  # 30 seconds
    )
    
    # Test connection
    health = client.health()
    
    print("✅ Connected to InfluxDB!")
    print(f"   Status: {health.status}")
    print(f"   Message: {health.message if hasattr(health, 'message') else 'OK'}")
    
    # Create write and query APIs
    write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()
    
    print("✓ Write API ready")
    print("✓ Query API ready")
    
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("\nTroubleshooting:")
    print("1. Check your token is correct")
    print("2. Verify the endpoint URL is accessible")
    print("3. Ensure your organization name is correct")

## Step 5: Write Sample Energy Data

Let's write some sample energy metrics to test the connection.

In [ ]:
import random

def write_energy_metrics(num_points=10):
    """
    Write sample energy metrics to InfluxDB
    """
    points = []
    base_time = datetime.utcnow() - timedelta(hours=1)
    
    for i in range(num_points):
        timestamp = base_time + timedelta(minutes=i * 6)
        
        point = (
            Point("energy_metrics")
            .tag("location", "datacenter-1")
            .tag("source", "sagemaker-test")
            .field("power_kw", round(random.uniform(50, 150), 2))
            .field("voltage", round(random.uniform(220, 240), 2))
            .field("current", round(random.uniform(10, 30), 2))
            .field("temperature_c", round(random.uniform(20, 35), 2))
            .time(timestamp, WritePrecision.NS)
        )
        points.append(point)
    
    try:
        write_api.write(bucket=INFLUXDB_BUCKET, org=INFLUXDB_ORG, record=points)
        print(f"✅ Wrote {num_points} energy metric points")
        print(f"   Measurement: energy_metrics")
        print(f"   Fields: power_kw, voltage, current, temperature_c")
        print(f"   Time range: {base_time.strftime('%H:%M')} to {timestamp.strftime('%H:%M')}")
        return True
    except Exception as e:
        print(f"❌ Write failed: {e}")
        return False

# Write 20 sample points
write_energy_metrics(20)

## Step 6: Write Sample Sensor Data

In [ ]:
def write_sensor_data(num_points=10):
    """
    Write sample sensor data to InfluxDB
    """
    points = []
    base_time = datetime.utcnow() - timedelta(hours=1)
    
    for i in range(num_points):
        timestamp = base_time + timedelta(minutes=i * 6)
        sensor_id = f"sensor-{random.randint(1, 3)}"
        
        point = (
            Point("sensor_data")
            .tag("sensor_id", sensor_id)
            .tag("source", "sagemaker-test")
            .field("temperature", round(random.uniform(18, 28), 2))
            .field("humidity", round(random.uniform(35, 65), 2))
            .field("pressure", round(random.uniform(990, 1015), 2))
            .time(timestamp, WritePrecision.NS)
        )
        points.append(point)
    
    try:
        write_api.write(bucket=INFLUXDB_BUCKET, org=INFLUXDB_ORG, record=points)
        print(f"✅ Wrote {num_points} sensor data points")
        print(f"   Measurement: sensor_data")
        print(f"   Fields: temperature, humidity, pressure")
        print(f"   Sensors: sensor-1, sensor-2, sensor-3")
        return True
    except Exception as e:
        print(f"❌ Write failed: {e}")
        return False

# Write 20 sample points
write_sensor_data(20)

## Step 7: Query Energy Metrics

In [ ]:
def query_energy_data(hours=2):
    """
    Query energy metrics from InfluxDB
    """
    flux_query = f'''
    from(bucket: "{INFLUXDB_BUCKET}")
      |> range(start: -{hours}h)
      |> filter(fn: (r) => r._measurement == "energy_metrics")
      |> filter(fn: (r) => r._field == "power_kw" or r._field == "temperature_c")
      |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
    '''
    
    try:
        tables = query_api.query(flux_query, org=INFLUXDB_ORG)
        
        # Convert to pandas DataFrame
        records = []
        for table in tables:
            for record in table.records:
                records.append({
                    'time': record.get_time(),
                    'location': record.values.get('location'),
                    'power_kw': record.values.get('power_kw'),
                    'temperature_c': record.values.get('temperature_c')
                })
        
        df = pd.DataFrame(records)
        print(f"✅ Retrieved {len(df)} energy records")
        return df
    except Exception as e:
        print(f"❌ Query failed: {e}")
        return pd.DataFrame()

# Query data
energy_df = query_energy_data(hours=2)

if not energy_df.empty:
    display(energy_df.head(10))
    print(f"\nData summary:")
    print(f"  Total records: {len(energy_df)}")
    print(f"  Average power: {energy_df['power_kw'].mean():.2f} kW")
    print(f"  Max power: {energy_df['power_kw'].max():.2f} kW")
    print(f"  Min power: {energy_df['power_kw'].min():.2f} kW")

## Step 8: Visualize Energy Data

In [ ]:
if not energy_df.empty:
    # Create power consumption chart
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=energy_df['time'],
        y=energy_df['power_kw'],
        mode='lines+markers',
        name='Power Consumption',
        line=dict(color='#3b82f6', width=2),
        marker=dict(size=6)
    ))
    
    fig.update_layout(
        title='Energy Consumption Over Time',
        xaxis_title='Time',
        yaxis_title='Power (kW)',
        hovermode='x unified',
        template='plotly_white',
        height=500,
        showlegend=True
    )
    
    fig.show()
else:
    print("No data to visualize. Make sure you ran the write operations above.")

## Step 9: Query Sensor Data

In [ ]:
def query_sensor_data(hours=2):
    """
    Query sensor data from InfluxDB
    """
    flux_query = f'''
    from(bucket: "{INFLUXDB_BUCKET}")
      |> range(start: -{hours}h)
      |> filter(fn: (r) => r._measurement == "sensor_data")
      |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
    '''
    
    try:
        tables = query_api.query(flux_query, org=INFLUXDB_ORG)
        
        records = []
        for table in tables:
            for record in table.records:
                records.append({
                    'time': record.get_time(),
                    'sensor_id': record.values.get('sensor_id'),
                    'temperature': record.values.get('temperature'),
                    'humidity': record.values.get('humidity'),
                    'pressure': record.values.get('pressure')
                })
        
        df = pd.DataFrame(records)
        print(f"✅ Retrieved {len(df)} sensor records")
        return df
    except Exception as e:
        print(f"❌ Query failed: {e}")
        return pd.DataFrame()

# Query sensor data
sensor_df = query_sensor_data(hours=2)

if not sensor_df.empty:
    display(sensor_df.head(10))
    print(f"\nData summary:")
    print(f"  Total records: {len(sensor_df)}")
    print(f"  Average temp: {sensor_df['temperature'].mean():.2f}°C")
    print(f"  Average humidity: {sensor_df['humidity'].mean():.1f}%")

## Step 10: Visualize Sensor Data

In [ ]:
if not sensor_df.empty:
    # Temperature by sensor
    fig = px.line(
        sensor_df,
        x='time',
        y='temperature',
        color='sensor_id',
        title='Sensor Temperature Readings',
        labels={'temperature': 'Temperature (°C)', 'time': 'Time', 'sensor_id': 'Sensor'},
        template='plotly_white',
        markers=True
    )
    
    fig.update_layout(height=500)
    fig.show()
    
    # Humidity chart
    fig2 = px.line(
        sensor_df,
        x='time',
        y='humidity',
        color='sensor_id',
        title='Sensor Humidity Readings',
        labels={'humidity': 'Humidity (%)', 'time': 'Time', 'sensor_id': 'Sensor'},
        template='plotly_white',
        markers=True
    )
    
    fig2.update_layout(height=500)
    fig2.show()
else:
    print("No sensor data to visualize.")

## Step 11: Close Connection

In [ ]:
# Close the client when done
client.close()
print("✓ InfluxDB connection closed")

## Summary

### What You Tested:
- ✅ Connected to AWS Timestream for InfluxDB
- ✅ Wrote energy metrics (power, voltage, current, temperature)
- ✅ Wrote sensor data (temperature, humidity, pressure)
- ✅ Queried data using Flux query language
- ✅ Visualized time-series data with Plotly

### Database Info:
- **Database:** superAppDB
- **Endpoint:** https://Lhk52q7uoe-lktzzbuyksah47.timestream-influxdb.us-east-1.on.aws
- **Organization:** superapp-org
- **Bucket:** test-bucket

### Next Steps:
1. Change the bucket name to match your actual buckets (e.g., `energy_metrics`, `sensor_data`)
2. Write your own custom data
3. Create more complex Flux queries
4. Build dashboards with your data
5. Integrate with your production applications (esapp, tsapp)

---

**Need help?** See `GET_INFLUXDB_TOKEN.md` for token setup or the main README for more information.